In [1]:
from numba import njit
from hftbacktest import (
    HftBacktest,
    FeedLatency,
    SquareProbQueueModel,
    Linear,
    Stat,
    BUY,
    SELL,
    GTX,
    ConstantLatency,
    PartialFillExchange
)

In [2]:
@njit
def simple_two_sided_quote(hbt, stat):
    max_position = 5
    half_spread = hbt.tick_size * 2
    skew = 1
    order_qty = 1
    last_order_id = -1
    order_id = 0
    print("Starting simple two-sided quote strategy")
    loop_count = 0
    # Checks every 0.1s
    prev_mid_price = None
    while hbt.elapse(100_000):
        loop_count += 1
        if loop_count % 1_000 == 0:
            print("Loop count: ", loop_count)
        if loop_count > 1_000:
            break
        # Clears cancelled, filled or expired orders.
        hbt.clear_inactive_orders()

        # Obtains the current mid-price and compute the reservation price.
        print("Best bid: ", hbt.best_bid)
        print("Best ask: ", hbt.best_ask)
        mid_price = (hbt.best_bid + hbt.best_ask) / 2.0
        print("Mid price: ", mid_price)
        if prev_mid_price is None:
            prev_mid_price = mid_price
            continue

        if mid_price < prev_mid_price:
            # Buy
            order_id += 1
            hbt.submit_buy_order(
                order_id,
                mid_price,
                order_qty,
                GTX
            )
            last_order_id = order_id
        else:
            # Sell
            order_id += 1
            hbt.submit_sell_order(
                order_id,
                mid_price,
                order_qty,
                GTX
            )
            last_order_id = order_id
        # reservation_price = mid_price - skew * hbt.position * hbt.tick_size

        # buy_order_price = reservation_price - half_spread
        # sell_order_price = reservation_price + half_spread

        # last_order_id = -1
        # # Cancel all outstanding orders
        # for order in hbt.orders.values():
        #     if order.cancellable:
        #         hbt.cancel(order.order_id)
        #         last_order_id = order.order_id

        # # All order requests are considered to be requested at the same time.
        # # Waits until one of the order cancellation responses is received.
        # if last_order_id >= 0:
        #     hbt.wait_order_response(last_order_id)

        # # Clears cancelled, filled or expired orders.
        # hbt.clear_inactive_orders()

        # if hbt.position < max_position:
        #     # Submits a new post-only limit bid order.
        #     order_id += 1
        #     hbt.submit_buy_order(
        #         order_id,
        #         buy_order_price,
        #         order_qty,
        #         GTX
        #     )
        #     last_order_id = order_id

        # if hbt.position > -max_position:
        #     # Submits a new post-only limit ask order.
        #     order_id += 1
        #     hbt.submit_sell_order(
        #         order_id,
        #         sell_order_price,
        #         order_qty,
        #         GTX
        #     )
        #     last_order_id = order_id

        # # All order requests are considered to be requested at the same time.
        # # Waits until one of the order responses is received.
        # if last_order_id >= 0:
        #     hbt.wait_order_response(last_order_id)

        # # Records the current state for stat calculation.
        stat.record(hbt)

In [3]:
# This backtest assumes market maker rebates.
# https://www.binance.com/en/support/announcement/binance-upgrades-usd%E2%93%A2-margined-futures-liquidity-provider-program-2023-04-04-01007356e6514df3811b0c80ab8c83bf
    
hbt = HftBacktest(
    [
        "/home/danny/hftbacktest/test_dbn.npz"
    ],
    tick_size=0.01,
    lot_size=1,
    maker_fee=0,
    taker_fee=0.0,
    order_latency=ConstantLatency(0.0001, 0.0001),
    queue_model=SquareProbQueueModel(),
    asset_type=Linear,
    exchange_model=PartialFillExchange,
    # snapshot='data/btcusdt_20230430_eod.npz'
)

stat = Stat(hbt)

Load /home/danny/hftbacktest/test_dbn.npz
loaded


In [4]:
simple_two_sided_quote(hbt, stat.recorder)

Starting simple two-sided quote strategy
Best bid:  -9.223372036854776e+16
Best ask:  9.223372036854776e+16
Mid price:  0.0
Best bid:  -9.223372036854776e+16
Best ask:  9.223372036854776e+16
Mid price:  0.0
Best bid:  -9.223372036854776e+16
Best ask:  9.223372036854776e+16
Mid price:  0.0
Best bid:  -9.223372036854776e+16
Best ask:  9.223372036854776e+16
Mid price:  0.0
Best bid:  -9.223372036854776e+16
Best ask:  9.223372036854776e+16
Mid price:  0.0
Best bid:  -9.223372036854776e+16
Best ask:  9.223372036854776e+16
Mid price:  0.0
Best bid:  -9.223372036854776e+16
Best ask:  9.223372036854776e+16
Mid price:  0.0
Best bid:  -9.223372036854776e+16
Best ask:  9.223372036854776e+16
Mid price:  0.0
Best bid:  -9.223372036854776e+16
Best ask:  9.223372036854776e+16
Mid price:  0.0
Best bid:  -9.223372036854776e+16
Best ask:  9.223372036854776e+16
Mid price:  0.0
Best bid:  -9.223372036854776e+16
Best ask:  9.223372036854776e+16
Mid price:  0.0
Best bid:  -9.223372036854776e+16
Best ask:  9

In [5]:
stat.summary(capital=2000)

IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
# print 
import sys 
import os 

# print total file size of all files in the directory 
